In [1]:
# Imports

import mlflow
import dagshub
import mlflow.sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# Setting up DagsHub

dagshub.init(repo_owner='SushrutGaikwad', repo_name='youtube-comments-analyzer', mlflow=True)

Accessing as SushrutGaikwad

Initialized MLflow to track repo "SushrutGaikwad/youtube-comments-analyzer"

Repository SushrutGaikwad/youtube-comments-analyzer initialized!

# Data

In [3]:
PREPROCESSED_DATA_PATH = "../data/processed/reddit_preprocessed.csv"
df = pd.read_csv(PREPROCESSED_DATA_PATH)
df.dropna(subset=["clean_comment"], inplace=True)
df.shape

(36662, 2)

# Running the experiment

In [4]:
# Setting experiment name

mlflow.set_experiment("Exp 2: BoW bigrams max_features")

<Experiment: artifact_location='mlflow-artifacts:/52fd7813915846e4b242cbb746adb4b7', creation_time=1748885351891, experiment_id='3', last_update_time=1748885351891, lifecycle_stage='active', name='Exp 2: BoW bigrams max_features', tags={}>

In [5]:
def run_experiment(max_features):
    ngram_range = (1, 2)  # bigrams

    vectorizer = CountVectorizer(
        ngram_range=ngram_range,
        max_features=max_features
    )

    X_train, X_test, y_train, y_test = train_test_split(
        df["clean_comment"],
        df["category"],
        test_size=0.2,
        random_state=42,
        stratify=df["category"]
    )
    X_train_vectorized = vectorizer.fit_transform(X_train)
    X_test_vectorized = vectorizer.transform(X_test)

    with mlflow.start_run() as run:
        # Tags
        mlflow.set_tag("mlflow.runName", f"BoW_bigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Description
        mlflow.set_tag(
            "description",
            f"RandomForest with BoW bigrams, max_features={max_features}."
        )

        # Logging vectorizer parameters
        mlflow.log_param("vectorizer_type", "BoW")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Logging random forest parameters
        n_estimators = 200
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initializing and training the model
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42
        )
        model.fit(X_train_vectorized, y_train)

        # Making predictions on the test set and logging metrics
        y_pred = model.predict(X_test_vectorized)

        # Logging accuracy
        accuracy = accuracy_score(
            y_true=y_test,
            y_pred=y_pred
        )
        mlflow.log_metric("accuracy", accuracy)

        # Logging classification report metrics
        classification_rep = classification_report(
            y_true=y_test,
            y_pred=y_pred,
            output_dict=True
        )
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}: {metric}", value)

        # Logging the confusion matrix
        conf_matrix = confusion_matrix(
            y_true=y_test,
            y_pred=y_pred,
        )
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion matrix: BoW bigrams, max_features={max_features}")
        plt.savefig(
            f"../reports/figures/confusion_matrix_BoW_bigrams_max_features_{max_features}.png"
        )
        mlflow.log_artifact(
            f"../reports/figures/confusion_matrix_BoW_bigrams_max_features_{max_features}.png"
        )
        plt.close()

        # Logging the model
        mlflow.sklearn.log_model(
            model,
            f"random_forest_model_bow_bigrams_max_features_{max_features}"
        )

In [6]:
max_features_vals = [
    1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000
]

for max_features in max_features_vals:
    run_experiment(max_features)

2025/06/02 23:54:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_bigrams_max_features_1000 at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/3/runs/ebfa1c24e85c4630b05a934d626f69df
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/3


2025/06/02 23:54:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_bigrams_max_features_2000 at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/3/runs/d544f1e6f96e43f1ba4a43f379054a92
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/3


2025/06/02 23:55:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run BoW_bigrams_max_features_8000 at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/3/runs/b3b7bc01538f410b836d56ec1461d409
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/3


2025/06/03 00:00:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_bigrams_max_features_9000 at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/3/runs/d7beb2a264e64aa3b591b8c28396da92
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/3


2025/06/03 00:00:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_bigrams_max_features_10000 at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/3/runs/1beb418d9b9d432dac2b8a757b958161
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/3
